### Группировка данных и оконные функции.

#### Задание ####

1) Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.
2) Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.
3) Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.
4) Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. 
    - 4.1) Выполните используя только group by.
    - 4.2) Выполните используя только оконные функции.
5) Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). 
    - 5.1) Напишите запрос для минимальной суммы.
    - 5.2) Напишите запрос для максимальной суммы.
6) Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.
7) Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях).

Так как в первом домашнем задании мы уже работали с этими данными и улучшили их структуру, я буду использовать созданную мной схему, а не исходную. Это кажется мне более логичным и правильным решением.

![title](Схема_0.png)

#### 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. ####

~~~~sql
SELECT 
    job_industry_category, 
    COUNT(job_industry_category)
FROM job_categories
GROUP BY 
    job_industry_category
ORDER BY 
    COUNT(job_industry_category) DESC;
~~~~

![title](Схема_1.png)

#### 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. ####

~~~~sql
SELECT 
    TO_CHAR(transactions.transaction_date, 'YYYY-MM') AS transaction_month,
    job_categories.job_industry_category,
    SUM(transactions.list_price) AS total_transaction
FROM transactions
    JOIN customers ON transactions.customer_id = customers.customer_id
    JOIN job_categories ON customers.job_id = job_categories.job_id
GROUP BY 
    transaction_month, 
    job_categories.job_industry_category
ORDER BY 
    transaction_month, 
    job_categories.job_industry_category;
~~~~

![title](Схема_2.png)

#### 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. ####

~~~~sql
SELECT 
    products.brand, 
    COUNT(transactions.transaction_id) AS count_transactions
FROM transactions
    JOIN customers ON transactions.customer_id = customers.customer_id
    JOIN products ON transactions.product_new_id = products.product_new_id 
    JOIN job_categories ON customers.job_id = job_categories.job_id
WHERE 
    (transactions.online_order = TRUE)
    AND (transactions.order_status = 'Approved')
    AND (job_categories.job_industry_category = 'IT')
GROUP BY 
    products.brand;
~~~~

![title](Схема_3.png)

#### 4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. ####

##### 4.1. Выполните используя только group by.

Насколько я понял функции SUM(), MIN(), MAX(), COUNT() не являются оконными без OVER().

~~~~sql
SELECT 
    customers.customer_id, 
    customers.first_name, 
    customers.last_name, 
    customers.dob,
    SUM(transactions.list_price) AS sum_transactions,
    MIN(transactions.list_price) AS min_transactions,
    MAX(transactions.list_price) AS max_transactions,
    COUNT(transactions.list_price) AS count_transactions
FROM transactions
    JOIN customers ON transactions.customer_id = customers.customer_id
GROUP BY 
    customers.customer_id, 
    customers.first_name, 
    customers.last_name, 
    customers.dob
ORDER BY 
    sum_transactions DESC,
    count_transactions DESC;
~~~~

![title](Схема_4.1.png)

##### 4.2. Выполните используя только оконные функции.

~~~~sql
WITH aggregated AS (
    SELECT
        customers.customer_id,
        customers.first_name,
        customers.last_name,
        customers.dob,
        SUM(transactions.list_price) OVER (PARTITION BY customers.customer_id) AS sum_transactions,
        MIN(transactions.list_price) OVER (PARTITION BY customers.customer_id) AS min_transactions,
        MAX(transactions.list_price) OVER (PARTITION BY customers.customer_id) AS max_transactions,
        COUNT(transactions.list_price) OVER (PARTITION BY customers.customer_id) AS count_transactions,
        ROW_NUMBER() OVER (PARTITION BY customers.customer_id ORDER BY transactions.transaction_date) AS rn
    FROM transactions
        JOIN customers ON transactions.customer_id = customers.customer_id
)
SELECT 
    customer_id, 
    first_name, 
    last_name, 
    dob,
    sum_transactions,
    min_transactions,
    max_transactions,
    count_transactions
FROM aggregated
WHERE 
    rn = 1
ORDER BY 
    sum_transactions DESC,
    count_transactions DESC;
~~~~

![title](Схема_4.2.png)

#### 5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). ####

##### 5.1. Напишите запрос для минимальной суммы.

~~~~sql
WITH aggregated AS (
    SELECT 
        customers.first_name,
        customers.last_name,
        SUM(transactions.list_price) AS min_sum_transactions
    FROM transactions
        JOIN customers ON transactions.customer_id = customers.customer_id
    GROUP BY 
        customers.first_name, 
        customers.last_name
)
SELECT 
    first_name, 
    last_name, 
    min_sum_transactions
FROM aggregated
WHERE 
    min_sum_transactions = (
        SELECT 
            MIN(min_sum_transactions) 
        FROM aggregated
    );
~~~~

![title](Схема_5.1.png)

##### 5.2. Напишите запрос для максимальной суммы.

~~~~sql
WITH aggregated AS (
    SELECT 
        customers.first_name,
        customers.last_name,
        SUM(transactions.list_price) AS max_sum_transactions
    FROM transactions
        JOIN customers ON transactions.customer_id = customers.customer_id
    GROUP BY 
        customers.first_name, 
        customers.last_name
)
SELECT 
    first_name, 
    last_name, 
    max_sum_transactions
FROM aggregated
WHERE 
    max_sum_transactions = (
        SELECT 
            MAX(max_sum_transactions) 
        FROM aggregated
    );
~~~~

![title](Схема_5.2.png)

#### 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. ####

~~~~sql
WITH aggregated AS (
    SELECT 
        transactions.transaction_id,
        transactions.customer_id,
        transactions.product_new_id,
        transactions.transaction_date,
        transactions.online_order,
        transactions.order_status,
        transactions.list_price,
        ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY transaction_date) AS rn
    FROM transactions
)
SELECT
    transaction_id,
    customer_id,
    product_new_id,
    transaction_date,
    online_order,
    order_status,
    list_price
FROM aggregated
WHERE 
    rn = 1;
~~~~

![title](Схема_6.png)

#### 7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях). ####

~~~~sql
WITH customer_max AS (
    SELECT 
        customer_id,
    MAX(gap) AS max_gap
    FROM (
        SELECT 
            customer_id,
            transaction_date - LAG(transaction_date) OVER (PARTITION BY customer_id ORDER BY transaction_date) AS gap
        FROM transactions
    ) AS gaps
    GROUP BY 
        customer_id
)
SELECT 
    customers.first_name, 
    customers.last_name, 
    job_categories.job_title
FROM customer_max
    JOIN customers ON customer_max.customer_id = customers.customer_id
    JOIN job_categories ON customers.job_id = job_categories.job_id
WHERE 
    customer_max.max_gap = (
    SELECT 
        MAX(max_gap) 
    FROM customer_max
    );
~~~~

![title](Схема_7.png)